In [1]:
import sys
if ".." not in sys.path:
    sys.path.append("..")

In [8]:
import os
import pickle
import warnings
from collections import defaultdict
from datetime import datetime

import numpy as np
import pandas as pd
from cobra.io import load_model, read_sbml_model
from matplotlib import pyplot as plt

from experiments.cue import plot_bge, plot_result
from experiments.experiment import Experiment
from experiments.fast_dFBA import (ConstantBounds, dFBA, make_bge_listener,
                                   make_boundary_listener, make_cue_listener,
                                   make_growth_rate_listener,
                                   make_shadow_price_listener,
                                   plot_shadow_prices)
from parameters.drawdown import *
from utils.cobra_utils import get_or_create_exchange
from utils.units import u

C_PER_GLUCOSE = 6
C_PER_ACETATE = 2

BIOMASS_ID = "Rpom_hwa_biomass"
DATA_FILE = "../data/clean/CUE2/dFBA.pkl"

# Load and set up model
model = read_sbml_model("../model/Rpom_05.xml")
ex_ace = get_or_create_exchange(model, "ACET[e]")

# Turn on maintenance
atpm = model.reactions.get_by_id("ATPM")
atpm.bounds = (25, 25)

# Load condition data
with open(DATA_FILE, "rb") as f:
    data = pickle.load(f)

In [9]:
data.keys()

dict_keys([(<Quantity(2.0, 'millimolar')>, <Quantity(0.0, 'millimolar')>), (<Quantity(0.0, 'millimolar')>, <Quantity(6.0, 'millimolar')>), (<Quantity(0.666666667, 'millimolar')>, <Quantity(4.0, 'millimolar')>), (<Quantity(1.33333333, 'millimolar')>, <Quantity(2.0, 'millimolar')>)])

In [18]:
g12a0 = [v for (g, a), v in data.items() if g > 0 and a == 0][0]
g0a12 = [v for (g, a), v in data.items() if g == 0 and a > 0][0]
g4a8 = [v for (g, a), v in data.items() if 0 < g.magnitude < 1 and a > 0][0]
g8a4 = [v for (g, a), v in data.items() if g.magnitude > 1 and a > 0][0]

In [21]:
g12a0["raw"].keys()

dict_keys(['raw_b_t', 'raw_b', 'raw_g_t', 'raw_g_s', 'raw_a_t', 'raw_a_s'])